<a href="https://colab.research.google.com/github/HungVLe/CMPE255/blob/main/Approximate_nearest_neighbor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Dataset**: 
https://github.com/groveco/content-engine/blob/master/sample-data.csv


**References**: 
https://github.com/eyaltrabelsi/my-notebooks/tree/master/Lectures/search_in_practice-approximate_nearest_neighbors

In [ ]:
import pandas as pd
import numpy as np
import re

import texthero as hero
from texthero import preprocessing

#**Load and pre-processing data**

In [ ]:
dataset = pd.read_csv('sample-data.csv')
dataset.head()

,id,description
0,1,Active classic boxers - There's a reason why o...
1,2,Active sport boxer briefs - Skinning up Glory ...
2,3,Active sport briefs - These superbreathable no...
3,4,"Alpine guide pants - Skin in, climb ice, switc..."
4,5,"Alpine wind jkt - On high ridges, steep ice an..."


##**Clean text**

In [ ]:
custom_pipeline = [preprocessing.fillna,
                   preprocessing.remove_whitespace,
                   preprocessing.remove_diacritics,
                   preprocessing.remove_punctuation,
                   preprocessing.remove_stopwords,
                   preprocessing.remove_digits,
                   preprocessing.lowercase
                  ]
def clean_html_tags(text):
    text = re.sub('<.*?>','',text)
    return text 

In [ ]:
dataset['description']=dataset['description'].apply(lambda x: clean_html_tags(x))
dataset['description'] = hero.clean(dataset['description'], custom_pipeline)
dataset['description'][0]

'active classic boxers   there   reason   boxers   cult favorite    keep  cool  especially  sticky situations  the quick drying  lightweight underwear takes  minimal space   travel pack  an exposed  brushed waistband offers next  skin softness  five panel construction   traditional boxer back   classic fit    functional fly  made      oz    recycled polyester  moisture wicking performance  inseam  size m          recyclable   common threads recycling program details   silky capilene   fabric  ultralight  breathable  quick  dry   exposed  brushed elastic waistband  comfort    panel construction  traditional boxer back  inseam  size m         fabric      oz     recycled polyester  gladiodor natural odor control   garment  recyclable   common threads recycling programweight    g      oz made  mexico '

##**Vectorizing the text data**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(analyzer='word',ngram_range=(1, 3),min_df=0,stop_words='english')
X_tfidf = tfidf.fit_transform(dataset['description'])
X_tfidf

<500x49811 sparse matrix of type '<class 'numpy.float64'>'
	with 124214 stored elements in Compressed Sparse Row format>

In [ ]:
vector = X_tfidf.toarray()
vector[0].shape[0]

49811

#**Approximate nearest neighbor search**

In [ ]:
import faiss    

##**Locality-sensitive hashing**

In [ ]:
class LSHIndex():
    def __init__(self, vectors, labels):
        self.dimention = vectors[0].shape[0]
        self.vectors = vectors.astype('float32')
        self.labels = labels


    def build(self, num_bits=8):
        self.index = faiss.IndexLSH(self.dimention,num_bits)
        self.index.add(self.vectors)
        
    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        return [self.labels[i] for i in indices[0]]

In [ ]:
%%time
lsh_index = LSHIndex(vector, dataset['id'])
lsh_index.build()

Wall time: 401 ms


In [ ]:
query_index = 1
print(f"The most simillar to item {dataset['id'][query_index]} are:")
items_list = lsh_index.query(np.array([vector[query_index]]).astype('float32'))
dataset.loc[dataset['id'].isin(items_list)]

The most simillar to item 2 are:


,id,description
1,2,active sport boxer briefs skinning glory re...
29,30,cotton board shorts a classic fabric form ...
38,39,elias sweatshirt the dirtbagger complete wa...
42,43,gi ii pants the travel tested quick drying ...
83,84,lw travel pack in lifelong search place ...
187,188,inter continental pants long you like fl...
272,273,morning glory tights this everyday essential...
290,291,print adour btm adours slightly scooped l...
312,313,kamala dress like lotus flower named k...
340,341,cap scoop this silky top designed desert...


##**Exhaustive Search**

In [ ]:
class BruteForceIndex():
    def __init__(self, vectors, labels):
        self.vectors = vectors.astype('float32')
        self.labels = labels
        self.index = faiss.IndexFlatL2(vectors[0].shape[0])
        self.index.add(self.vectors)
        
    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        return [self.labels[i] for i in indices[0]]

In [ ]:
%%time
es_index = BruteForceIndex(vector, dataset['id'])

Wall time: 97.2 ms


In [ ]:
query_index = 1
print(f"The most simillar to item {dataset['id'][query_index]} are:")
items_list = es_index.query(np.array([vector[query_index]]).astype('float32'))
dataset.loc[dataset['id'].isin(items_list)]

The most simillar to item 2 are:


,id,description
0,1,active classic boxers there reason boxer...
1,2,active sport boxer briefs skinning glory re...
2,3,active sport briefs these superbreathable f...
18,19,cap boxer briefs on bivy belay form fit...
164,165,barely bikini better going commando petal...
298,299,active boy shorts we worn versatile femin...
299,300,active briefs whether beating heat bali ...
317,318,barely hipster the barely hipster form fitt...
493,494,active boxer briefs a fuss travel companion...
494,495,active briefs these featherweight quick wic...


##**Product Quantization**

In [ ]:
class PQIndex():
    def __init__(self, vectors, labels):
        self.dimention = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels


    def build(self, number_of_partition=8, search_in_x_partitions=2, subvector_size=8):
        m = 3
        n_bits = 8
        self.index = faiss.IndexPQ (self.dimention, m, n_bits)
        self.index.train(self.vectors)
        self.index.add(self.vectors)
        
    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        return [self.labels[i] for i in indices[0]]

The number of bits n_bits (bits allocated per subquantizer) must be equal to 8, 12 or 16. The dimension d should be a multiple of m (number of subquantizers). Therefore, we need to apply batch here since the vector dimension is 49811, which is a prime number.

In [ ]:
batch_vector = []
for idx, arr in enumerate(vector):
    batch_vector.append(list(np.append(arr, 0.0)))

In [ ]:
%%time
batch_vector = np.array(batch_vector)
pq_index = PQIndex(batch_vector, dataset['id'])
pq_index.build()

Wall time: 3min 44s


In [ ]:
query_index = 1
print(f"The most simillar to item {dataset['id'][query_index]} are:")
items_list = pq_index.query(np.array([batch_vector[query_index]]).astype('float32'))
dataset.loc[dataset['id'].isin(items_list)]

The most simillar to item 2 are:


,id,description
1,2,active sport boxer briefs skinning glory re...
2,3,active sport briefs these superbreathable f...
10,11,baby sunshade top soft stretchy polyester f...
44,45,girl boardie capris built two week camping...
58,59,borderless gi shorts these shorts mud san...
226,227,s sol patrol shirt a shirt impossible insu...
256,257,sol patrol shirt go ahead go bone fishing ...
361,362,s sol patrol shirt in theory chasing indon...
398,399,marlwalker pants veterans tropics know lon...
477,478,l sol patrol shirt two week boat trips ind...


##**Trees and Graphs**

In [ ]:
import annoy

In [ ]:
class TreeGraphIndex():
    def __init__(self, vectors, labels):
        self.dimention = vectors[0].shape[0]
        self.vectors = vectors.astype('float32')
        self.labels = labels


    def build(self, number_of_trees=5):
        self.index = annoy.AnnoyIndex(self.dimention, metric='angular')
        for i, vec in enumerate(self.vectors):
            self.index.add_item(i, vec.tolist())
        self.index.build(number_of_trees)
        
    def query(self, vector, k=10):
        indices = self.index.get_nns_by_vector(vector.tolist(), k)
        return [self.labels[i] for i in indices]

In [ ]:
%%time
tg_index = TreeGraphIndex(vector, dataset['id'])
tg_index.build()

Wall time: 1.8 s


In [ ]:
query_index = 1
print(f"The most simillar to item {dataset['id'][query_index]} are:")
items_list = tg_index.query(vector[query_index])
dataset.loc[dataset['id'].isin(items_list)]

The most simillar to item 2 are:


,id,description
0,1,active classic boxers there reason boxer...
1,2,active sport boxer briefs skinning glory re...
2,3,active sport briefs these superbreathable f...
18,19,cap boxer briefs on bivy belay form fit...
164,165,barely bikini better going commando petal...
298,299,active boy shorts we worn versatile femin...
299,300,active briefs whether beating heat bali ...
317,318,barely hipster the barely hipster form fitt...
493,494,active boxer briefs a fuss travel companion...
494,495,active briefs these featherweight quick wic...


##**Hierarchical Navigable Small Worlds (HNSW)**

In [ ]:
import nmslib

In [ ]:
class HNSWIndex():
    def __init__(self, vectors, labels):
        self.dimention = vectors[0].shape[0]
        self.vectors = vectors.astype('float32')
        self.labels = labels

    def build(self):
        self.index = nmslib.init(method='hnsw', space='cosinesimil')
        self.index.addDataPointBatch(self.vectors)
        self.index.createIndex({'post': 2})
        
    def query(self, vector, k=10):
        indices = self.index.knnQuery(vector, k=k)
        return [self.labels[i] for i in indices[0]]

In [ ]:
%%time
hnsw_index = HNSWIndex(vector, dataset['id'])
hnsw_index.build()

Wall time: 6.81 s


In [ ]:
query_index = 1
print(f"The most simillar to item {dataset['id'][query_index]} are:")
items_list = hnsw_index.query(vector[query_index])
dataset.loc[dataset['id'].isin(items_list)]

The most simillar to item 2 are:


,id,description
0,1,active classic boxers there reason boxer...
1,2,active sport boxer briefs skinning glory re...
2,3,active sport briefs these superbreathable f...
18,19,cap boxer briefs on bivy belay form fit...
164,165,barely bikini better going commando petal...
298,299,active boy shorts we worn versatile femin...
299,300,active briefs whether beating heat bali ...
317,318,barely hipster the barely hipster form fitt...
493,494,active boxer briefs a fuss travel companion...
494,495,active briefs these featherweight quick wic...


#**Summary**

Locality-sensitive hashing 
* Library : faiss.IndexLSH
* Wall time : 401 ms 

Exhaustive Search
* Library : faiss.IndexFlatL2
* Wall time : 97.2 ms **(fastest)**

Product Quantization 
* Library : faiss.IndexPQ
* Wall time : 3min 44s **(longest)**

Trees and Graphs 
* Library : annoy.AnnoyIndex
* Wall time : 1.8 s

Hierarchical Navigable Small Worlds (HNSW) 
* Library : nmslib.init(method='hnsw', space='cosinesimil'))
* Wall time : 6.81 s
